In [1]:
import pandas as pd

In [2]:
from sklearn.metrics import roc_auc_score, log_loss

In [3]:
path = "~/Dropbox_RSM/teaching/big-data-22/grading/assignment-2/predictions"
path_data = "~/Dropbox_RSM/teaching/big-data-22/data/dr-s/private"

In [4]:
truth = pd.read_parquet(f"{path_data}/truth-no-coupon.parquet")

In [5]:
files = [
    "g4.parquet",
    "g1.parquet",
    "g2.csv",
    "g3.parquet",
    "g5.parquet",
    "g6.parquet",
    "g7.parquet",
]

In [9]:
results = {}

for f in files:
    group, filetype = f.split(".")
    if filetype == "csv":
        print(group)
        res_g = pd.read_csv(f"{path}/{f}")
    else:
        res_g = pd.read_parquet(f"{path}/{f}")
    if group == "g2":
        print(group)
        res_g.columns = ["week", "customer", "product", "probability"]
    if group == "g3":
        #probs = res_g["Logistic Regression predictions"].values
        #probs = res_g["Random Forest predictions"].values
        probs = res_g["Boosted Trees predictions"].values
        res_g = results["g1"].copy()
        res_g["probability"] = probs
        print(group)
    if group == "g7":
        print(group)
        res_g.columns = ["customer", "product", "probability"]
        res_g["week"] = 90
        res_g = res_g[["week", "customer", "product", "probability"]]
    results[group] = res_g

out = []

for k, v in results.items():
    tmp = pd.merge(truth, v, on=['week', 'customer', 'product'], how="left")
    tmp = tmp.fillna(0)
    out.append((k, log_loss(tmp["y"], tmp["probability"]), roc_auc_score(tmp["y"], tmp["probability"])))

pd.DataFrame(out, columns=["group", "loss", "auc"]).sort_values("loss")

g2
g2
g3
g7


,group,loss,auc
3,g3,0.076778,0.944311
0,g4,0.076879,0.945652
4,g5,0.080698,0.938894
1,g1,0.098571,0.944366
5,g6,0.107859,0.917175
2,g2,0.784077,0.519777
6,g7,0.980702,0.498382


In [7]:
truth2 = results["g7"][["week", "customer", "product"]].merge(truth, on=["week", "customer", "product"])

out = []

for k, v in results.items():
    tmp = pd.merge(truth2, v, on=['week', 'customer', 'product'], how="left")
    tmp = tmp.fillna(0)
    out.append((k, log_loss(tmp["y"], tmp["probability"]), roc_auc_score(tmp["y"], tmp["probability"])))

pd.DataFrame(out, columns=["group", "loss", "auc"]).sort_values("loss")

,group,loss,auc
3,g3,0.074487,0.944184
0,g4,0.074607,0.945335
4,g5,0.077768,0.938843
1,g1,0.093221,0.944253
5,g6,0.106561,0.914317
6,g7,0.124852,0.566615
2,g2,0.786502,0.514618
